In [1]:
import os
# Change root directory
os.chdir("../src/")


In [ ]:
from agent_design_pattern.agent import AgentMessage, LLMChain, IAgent
from agent_design_pattern.orchestration import SequentialAgent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

In [3]:
class CasualOllamaChain(LLMChain):
    def __init__(self, model, system_prompt, user_prompt_template = "{query}", **kargs):
        super().__init__()
        self.llm = ChatOllama(model=model, reasoning=False, **kargs) if isinstance(model, str) else model
        self.prompt = ChatPromptTemplate([
            ("system", system_prompt),
            ("human", user_prompt_template)
        ])
        self.chain = self.prompt | self.llm | StrOutputParser()

    def invoke(self, message: AgentMessage, **kwargs) -> AgentMessage:
        response = self.chain.invoke(message.to_dict(), **kwargs)
        print(response)
        # Snippet to remove thinking
        if "</think>" in response and "<think>" not in response:
            response = "<think>" + response
        response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
        message.response = response

        message.execution_result = "success"
        return message

## Define the agent

Create a simple agent for demonstration

In [4]:
class FunctionImplementationAgent(IAgent):
    def __init__(self, llm_chain: LLMChain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.llm_chain = llm_chain

    def execute(self, message: AgentMessage, **kwargs):
        return self.llm_chain.invoke(message, **kwargs)

In [ ]:
code_system_prompt = """You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Just the implementation, DO NOT comment anything else"""
code_user_prompt = "{query}"
code_chain = CasualOllamaChain("codegemma:2b-v1.1-q8_0-16384", code_system_prompt, code_user_prompt)

doc_system_prompt = """You are a great developer avocate.
You task is to write a document and in-code explain for the function.
Just the comment and explanation, DO NOT change the logic of the provided function.
The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code"""
doc_user_prompt = """Input query: {query}

Function implementation: {response}
"""
doc_chain = CasualOllamaChain("mistral:7b-instruct-v0.3-q4_K_S", doc_system_prompt, doc_user_prompt)

review_system_prompt = """You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain code, inline code comment and docstring, nothing else."""
review_user_prompt = """Input query: {query}

Function implementation: {response}
"""
review_chain = CasualOllamaChain("qwen3:4b-thinking-2507-q4_K_M", review_system_prompt, review_user_prompt)

def state_callback(message: str):
    print(message)

code_agent = FunctionImplementationAgent(code_chain, state_change_callback=state_callback, name="Code Agent")
doc_agent = FunctionImplementationAgent(doc_chain, state_change_callback=state_callback, name="Docstring Agent")
review_agent = FunctionImplementationAgent(review_chain, state_change_callback=state_callback, name="Review Agent")
agent = SequentialAgent([code_agent, doc_agent, review_agent], state_change_callback=state_callback, name="Sequential Agent")

In [8]:
query = "Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array."
message = agent.execute(AgentMessage(query=query))

print(message)
print(message.response)

running agent Code Agent
ipynb

running agent Docstring Agent
 Here is a Python function that finds the largest Fibonacci number less than or equal to a given limit, using an efficient iterative approach. The function is documented with comments and docstrings.

```python
def find_biggest_fib(limit):
    """
    Finds the largest Fibonacci number less than or equal to the given limit.

    Args:
        limit (int): An integer representing the maximum value for the search.

    Returns:
        The largest Fibonacci number less than or equal to the input limit.

    Example:
        >>> find_biggest_fib(20)
        13
        >>> find_biggest_fib(58)
        55
    """

    fibs = [0, 1]
    current, next = 0, 1

    while True:
        if current + next > limit:
            break
        fibs.append(current + next)
        current, next = next, current + next

    return max(fibs)
```

This function first initializes a list named `fibs` with the first two Fibonacci numbers (0 and 1). 